In [1]:
# Install necessary packages if not already installed
!pip install opencv-python numpy scikit-image pywavelets leidenalg

import os
import cv2
import numpy as np
from skimage import feature
from pywt import dwt2
from leidenalg import find_partition
import time
import psutil
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.9 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def calculate_CCFV(color_feature, edge_feature, wavelet_feature):
    # Step 1: Initialization
    X = color_feature.flatten()  # Color feature vector
    Y = np.concatenate([edge_feature.flatten(), wavelet_feature.flatten()])  # Combined feature vector

    # Step 2: Covariance Matrices
    n = len(X)
    mean_X = np.mean(X)
    mean_Y = np.mean(Y)

    Sxx = np.cov(X, rowvar=False)
    Syy = np.cov(Y, rowvar=False)
    Sxy = np.cov(X, Y, rowvar=False)

    # Step 3: Compute Transformation Matrices G1 and G2
    G1 = np.linalg.inv(np.sqrt(Sxx)).dot(Sxy).dot(np.linalg.inv(Syy)).dot(Sxy.T).dot(np.linalg.inv(np.sqrt(Sxx)))
    G2 = np.linalg.inv(np.sqrt(Syy)).dot(Sxy.T).dot(np.linalg.inv(Sxx)).dot(Sxy).dot(np.linalg.inv(np.sqrt(Syy)))

    # Step 4: Compute Eigen Vectors and Rank
    _, u = np.linalg.eig(G1)
    _, v = np.linalg.eig(G2)
    r = np.linalg.matrix_rank(Sxy)

    # Step 5: Choose Canonical Variables
    d = 100  # Choose the first d pairs of eigen vectors
    Wx = np.linalg.inv(np.sqrt(Sxx)).dot(u[:, :d])
    Wy = np.linalg.inv(np.sqrt(Syy)).dot(v[:, :d])

    # Step 6: Compute Canonically Correlated Feature Vector (CCFV)
    CCFV = (Wx.T.dot(X) + Wy.T.dot(Y)).reshape(-1, 1)

    return CCFV

def extract_frames(video_path):
    # Create a directory to store extracted frames
    frames_dir = "/content/gdrive/MyDrive/Video-Keyframe-Extraction-through-Canonical-Correlation-Analysis-and-Graph-Modularity-main/Test Results 03/extracted_frames"
    os.makedirs(frames_dir, exist_ok=True)

    # Read the video and extract frames
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    for frame_number in range(frame_count):
        ret, frame = video.read()
        if ret:
            frame_path = os.path.join(frames_dir, f"frame_{frame_number}.jpg")
            cv2.imwrite(frame_path, frame)

    video.release()

    return frames_dir

def extract_features(frames):
    color_features = []
    edge_features = []
    wavelet_features = []

    for frame in frames:
        # Extract Color feature (for example, using histogram)
        color_feature = cv2.calcHist([frame], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])

        # Extract Edge feature (for example, using Canny edge detector)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edge_feature = feature.canny(gray_frame)

        # Extract Wavelet feature (for example, using Haar wavelet transform)
        coeffs = dwt2(gray_frame, 'haar')
        wavelet_feature = np.concatenate((coeffs[0].flatten(), coeffs[1][0].flatten(), coeffs[1][1].flatten(), coeffs[1][2].flatten()))

        color_features.append(color_feature)
        edge_features.append(edge_feature)
        wavelet_features.append(wavelet_feature)

    return np.array(color_features), np.array(edge_features), np.array(wavelet_features)

def identify_keyframes(frames_dir, threshold=0.9):
    # Get the list of frame files
    frame_files = sorted(os.listdir(frames_dir))

    if not frame_files:
        raise ValueError("No frames found in the frames directory")

    # Initialize keyframes list
    keyframes = [frame_files[0]]  # Always add the first frame as a keyframe

    for i in range(1, len(frame_files)):
        current_frame_path = os.path.join(frames_dir, frame_files[i])
        prev_frame_path = os.path.join(frames_dir, frame_files[i - 1])

        # Read frames
        current_frame = cv2.imread(current_frame_path)
        prev_frame = cv2.imread(prev_frame_path)

        # Extract features
        current_color, current_edge, current_wavelet = extract_features([current_frame])
        prev_color, prev_edge, prev_wavelet = extract_features([prev_frame])

        # Calculate Canonically Correlated Feature Vector (CCFV)
        ccfv = calculate_CCFV(current_color, current_edge, current_wavelet)
        prev_ccfv = calculate_CCFV(prev_color, prev_edge, prev_wavelet)

        # Calculate Euclidean distance between CCFVs
        distance = np.linalg.norm(ccfv - prev_ccfv)

        # Check if the distance is below the threshold
        if distance > threshold:
            keyframes.append(frame_files[i])

    return keyframes

def summarize_video(video_path, num_frames, batch_size=100):
    # Extract frames from the video
    frames_dir = extract_frames(video_path)

    # Identify keyframes based on Multi-feature Fusion algorithm
    keyframes = identify_keyframes(frames_dir)

    # Create the "process_frames" directory and move keyframes there
    create_process_frames_directory(keyframes, frames_dir)

    # Perform clustering using Leiden graph modularity clustering algorithm
    labels = leiden_cluster(frames_dir, keyframes, num_frames, batch_size)

    # Get selected frames based on clustering labels
    selected_frames = get_selected_frames(frames_dir, keyframes, labels)

    return selected_frames

def create_process_frames_directory(keyframes, frames_dir):
    # Create a directory to store the identified keyframes
    process_frames_dir = os.path.join(frames_dir, "process_frames")
    os.makedirs(process_frames_dir, exist_ok=True)

    # Copy keyframes to the process_frames directory
    for keyframe in keyframes:
        source_path = os.path.join(frames_dir, keyframe)
        target_path = os.path.join(process_frames_dir, keyframe)
        try:
            os.rename(source_path, target_path)
        except FileNotFoundError:
            continue

def leiden_cluster(frames_dir, keyframes, num_frames, batch_size):
    # Perform clustering to obtain the labels
    labels = []

    # Process frames in batches
    for i in range(0, len(keyframes), batch_size):
        batch_frames = []
        for j in range(i, min(i + batch_size, len(keyframes))):
            frame_path = os.path.join(frames_dir, "process_frames", keyframes[j])
            frame = cv2.imread(frame_path)
            batch_frames.append(frame)

        # Extract features from the batch of frames
        color_features, edge_features, wavelet_features = extract_features(batch_frames)

        # Calculate Canonically Correlated Feature Vectors (CCFVs)
        ccfvs = np.array([calculate_CCFV(color, edge, wavelet) for color, edge, wavelet in
                          zip(color_features, edge_features, wavelet_features)])

        # Perform clustering using Leiden algorithm
        partition = find_partition(ccfvs, resolution_parameter=1.0)

        # Append cluster labels to the overall labels list
        labels.extend(partition)

    return labels

def get_selected_frames(frames_dir, keyframes, labels):
    selected_frames = []

    # Iterate over keyframes and corresponding labels
    for keyframe, label in zip(keyframes, labels):
        if label == 0:  # Select frames associated with the specific cluster (label)
            selected_frames.append(os.path.join(frames_dir, "process_frames", keyframe))

    return selected_frames

def create_summarized_video(input_video_path, output_video_path, selected_frames):
    # Get the video properties from the input video
    input_video = cv2.VideoCapture(input_video_path)
    frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(input_video.get(cv2.CAP_PROP_FPS))
    video_codec = cv2.VideoWriter_fourcc(*"mp4v")

    # Create the output video writer
    output_video = cv2.VideoWriter(output_video_path, video_codec, fps, (frame_width, frame_height))

    # Read the selected frames and write them to the output video
    for frame_path in selected_frames:
        frame = cv2.imread(frame_path)
        output_video.write(frame)

    # Release resources
    input_video.release()
    output_video.release()

def get_time_complexity():
    start_time = time.time()
    # Input video path
    input_video_path = '/content/gdrive/MyDrive/Video-Keyframe-Extraction-through-Canonical-Correlation-Analysis-and-Graph-Modularity-main/Test Results 03/Surveilance_camera_test03.mp4'

    # Specify the desired number of resultant frames
    num_frames = 10

    try:
        # Summarize the video and obtain selected frames
        selected_frames = summarize_video(input_video_path, num_frames)

        # Output video path
        output_video_path = '/content/gdrive/MyDrive/Video-Keyframe-Extraction-through-Canonical-Correlation-Analysis-and-Graph-Modularity-main/Test Results 03/Summarized_Surveilance_camera_test03.mp4'

        # Create the summarized output video
        create_summarized_video(input_video_path, output_video_path, selected_frames)

        print("Video summarization completed successfully!")
    except ValueError as e:
        print(f"Video summarization failed: {str(e)}")

    elapsed_time = time.time() - start_time
    return elapsed_time

def get_space_complexity():
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss  # in bytes
    memory_usage_mb = memory_usage / (1024 ** 2)  # convert to megabytes
    return memory_usage_mb

In [4]:
def main():
    time_complexity = get_time_complexity()
    space_complexity = get_space_complexity()

    print(f"Overall Time Complexity: {time_complexity} seconds")
    print(f"Overall Space Complexity: {space_complexity} MB")

if __name__ == "__main__":
    main()

Video summarization failed: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 16777216 and the array at index 1 has size 460800
Overall Time Complexity: 45.601526975631714 seconds
Overall Space Complexity: 187.76171875 MB
